## Data files used below are sourced from
https://cricsheet.org/downloads/india.zip
    
manually filtered for IND-AUS matches

In [1]:
from ruamel.yaml import YAML
import pandas as pd
import numpy as np
import os, glob
from pathlib import Path

# Team data sourced from https://www.cricbuzz.com/cricket-series/2773/australia-tour-of-india-2019/squads
india_team = ['R Sharma', 'S Dhawan', 'V Kohli', 'AT Rayudu', 'KL Rahul', 'R Pant', 'MS Dhoni', 'V Shankar', 'Kuldeep Yadav',
           'S Kaul', 'B Kumar', 'Mohammed Shami', 'JJ Bumrah', 'Y Chahal', 'RA Jadeja' ]

aus_team = ['AJ Finch', 'A Carey', 'UT Khawaja', 'SE Marsh', 'DA Short', 'PSP Handscomb', 'M Stoinis', 'GJ Maxwell', 'KW Richardson',
        'NM Coulter-Nile', 'J Behrendorff', 'NM Lyon', 'A Zampa']

## Function get_info_from_delivery() extracts desired data from a delivery record

In [2]:
def get_info_from_delivery(rows, yamlfile):
    tmp_list = []
    for row in rows:
        for ball_counter, val in row.items():
            tmp_dict = {}
            tmp_dict['filename'] = yamlfile
            tmp_dict['batsman']=val['batsman']
            tmp_dict['bowler']=val['bowler']
            tmp_dict['runs']=val['runs']['batsman']
            
            for key in val.keys():
                if (key == 'wicket'):
                    tmp_dict['wicket_type'] = val['wicket']['kind']
                    for w_key in val['wicket'].keys():
                        if (w_key == 'fielders'):
                            tmp_dict['fielder'] = val['wicket']['fielders'][0]
                    tmp_dict['player_out']=val['wicket']['player_out']
            tmp_list.append(tmp_dict)
    return(tmp_list)

## Parse all YAML files and build custom dictionary with fields of interest to be used as initializer for dataframe. Filename is included in dictionary for traceability.

### ODI match data from 2008 and later have been used. Other data is discarded.

In [3]:
player_df_init_list = []
winner_df_init_list = []
unused_files = []

#p=Path(r'/home/ec2-user/UpGrad_IPL/Datafiles/IND-AUS')
p=Path(r'./Datafiles/IND-AUS')

filelist = list(p.glob( "**/*.yaml" ))

yaml=YAML(typ='safe')

for yamlfile in filelist:
    datafile = open(yamlfile, 'r')
    mydict=yaml.load(datafile)
    if ( mydict['info']['dates'][0].year > 2008 ):
        if ( mydict['info']['match_type'] == 'ODI' ):
            if ( mydict['info']['gender'] == 'male' ):
                if (len(mydict['innings']) > 0):
                    if ( 'result' not in mydict['info']['outcome'].keys() ):
                        tmp_dict = {}
                        tmp_dict['country'] = mydict['info']['outcome']['winner']
                        winner_df_init_list.append(tmp_dict)
                    for inn in mydict['innings']:
                        idx = 0
                        for inn_key in ['1st innings', '2nd innings']:
                            if (inn_key in inn.keys()):
                                print('Fetching data from: ', str( yamlfile ) )
                                player_df_init_list += get_info_from_delivery( inn[inn_key]['deliveries'], str( yamlfile.name ) )
                            idx += 1
                else:
                    unused_files.append( str(yamlfile) + ': No innings data' )
            else:
                unused_files.append( str(yamlfile) + ': Female team' )
        else:
            unused_files.append( str(yamlfile) + ': Match type: ' + mydict['info']['match_type'] )
    else:
        unused_files.append( str(yamlfile) + ': Invalid year: ' + str( mydict['info']['dates'][0].year ) )

print('\n\nFollowing files were not used\n')
print(*unused_files, sep='\n')

Fetching data from:  Datafiles\IND-AUS\415281.yaml
Fetching data from:  Datafiles\IND-AUS\416236.yaml
Fetching data from:  Datafiles\IND-AUS\416236.yaml
Fetching data from:  Datafiles\IND-AUS\416237.yaml
Fetching data from:  Datafiles\IND-AUS\416237.yaml
Fetching data from:  Datafiles\IND-AUS\416238.yaml
Fetching data from:  Datafiles\IND-AUS\416238.yaml
Fetching data from:  Datafiles\IND-AUS\416239.yaml
Fetching data from:  Datafiles\IND-AUS\416239.yaml
Fetching data from:  Datafiles\IND-AUS\416240.yaml
Fetching data from:  Datafiles\IND-AUS\416240.yaml
Fetching data from:  Datafiles\IND-AUS\416241.yaml
Fetching data from:  Datafiles\IND-AUS\416241.yaml
Fetching data from:  Datafiles\IND-AUS\433601.yaml
Fetching data from:  Datafiles\IND-AUS\433601.yaml
Fetching data from:  Datafiles\IND-AUS\464529.yaml
Fetching data from:  Datafiles\IND-AUS\464529.yaml
Fetching data from:  Datafiles\IND-AUS\518956.yaml
Fetching data from:  Datafiles\IND-AUS\518956.yaml
Fetching data from:  Datafiles\

## Replace '( sub )' in player names. In data files, it has been used to indicate substitute.

In [4]:
tmp_df = pd.DataFrame(player_df_init_list)
tmp_df['fielder'].replace(to_replace='([\a\.\ ]*)( \(sub\))', value=r'\1', regex=True, inplace=True)
tmp_df['batsman'].replace(to_replace='([\a\.\ ]*)( \(sub\))', value=r'\1', regex=True, inplace=True)
tmp_df['bowler'].replace(to_replace='([\a\.\ ]*)( \(sub\))', value=r'\1', regex=True, inplace=True)
tmp_df['player_out'].replace(to_replace='([\a\.\ ]*)( \(sub\))', value=r'\1', regex=True, inplace=True)

## Check data frame looks OK

In [101]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16290 entries, 0 to 16289
Data columns (total 7 columns):
batsman        16290 non-null object
bowler         16290 non-null object
fielder        280 non-null object
filename       16290 non-null object
player_out     384 non-null object
runs           16290 non-null int64
wicket_type    384 non-null object
dtypes: int64(1), object(6)
memory usage: 890.9+ KB


## Create run rate dataframe for batsmen

In [5]:
batsman_df = tmp_df.groupby(['batsman', 'filename']).sum().sort_values(by=['batsman', 'filename']).reset_index() \
    .groupby(['batsman'])['runs'].agg([np.mean, np.size, np.sum]).reset_index()\
    .sort_values(by=['mean', 'size'], ascending=False)\
    .rename(columns={'sum':'score','size':'match_count', 'mean':'average_runs_per_match'})

## Create wicket taker data frame for bowlers

In [6]:
bowler_df = tmp_df[ ~tmp_df['player_out'].isnull() & tmp_df['fielder'].isnull() ].groupby(['bowler', 'filename'])\
    ['batsman'].agg(np.size).reset_index()\
    .groupby(['bowler'])['batsman'].agg([ np.mean, np.size, np.sum ]).reset_index()\
    .sort_values(by=['mean', 'size'], ascending=False).rename(columns={'size':'match_count', 'sum':'wickets', 'mean':'avg_wkt_per_match'})

## Create wicket taker data frame for fielders

In [104]:
fielder_df = tmp_df[ ~(tmp_df['player_out'].isnull() & tmp_df['fielder'].isnull()) ].groupby(['fielder', 'filename'])\
    ['batsman'].agg(np.size).reset_index()\
    .groupby(['fielder'])['batsman'].agg([ np.mean, np.size, np.sum ]).reset_index()\
    .sort_values(by=['mean', 'sum'], ascending=False).rename(columns={'size':'match_count', 'sum':'wickets', 'mean':'avg_wkt_per_match'})

## Check highest average score for batsmen in 2019 IND/AUS teams where matches played by batsman is more than average numbers of matches played. This ensures consistency in performance is accounted for. 

In [105]:
batsman_df[ batsman_df['batsman'].isin( india_team + aus_team ) &\
           ( batsman_df['match_count'] > batsman_df['match_count'].mean() ) ].head()

,batsman,average_runs_per_match,match_count,score
94,V Kohli,47.708333,24,1145
4,AJ Finch,44.615385,13,580
31,GJ Maxwell,38.333333,12,460
81,S Dhawan,38.176471,17,649
60,MS Dhoni,35.960000,25,899


## Check highest average score for bowlers in 2019 IND/AUS teams where matches played by bowler is more than average numbers of matches played. This ensures consistency in performance is accounted for. 

In [106]:
bowler_df[ bowler_df['bowler'].isin(aus_team + india_team) &\
           ( bowler_df['match_count'] > bowler_df['match_count'].mean() ) ].head()

,bowler,avg_wkt_per_match,match_count,wickets
19,Mohammed Shami,1.666667,3,5
26,RA Jadeja,1.285714,7,9


## Check highest average score for fielders in 2019 IND/AUS teams where matches played by fielder is more than average numbers of matches played. This ensures consistency in performance is accounted for. 

In [107]:
fielder_df[ fielder_df['fielder'].isin(aus_team + india_team) &\
           ( fielder_df['match_count'] > fielder_df['match_count'].mean() ) ].head()

,fielder,avg_wkt_per_match,match_count,wickets
38,MS Dhoni,1.866667,15,28
18,GJ Maxwell,1.600000,5,8
67,V Kohli,1.214286,14,17
55,S Dhawan,1.142857,7,8
50,RA Jadeja,1.125000,8,9


## Create dataframe for match winners

In [108]:
match_df = pd.DataFrame(winner_df_init_list)

### Compare number of wins by both countries

In [109]:
match_df.groupby('country').size().reset_index()

,country,0
0,Australia,15
1,India,12
